<a href="https://colab.research.google.com/github/moustafa-7/ECG-Project/blob/master/Cardiac_Arrhythmia_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install wfdb
!pip install pywavelets

     |████████████████████████████████| 102kB 5.9MB/s 
     |████████████████████████████████| 163kB 43.0MB/s 
  Created wheel for wfdb: filename=wfdb-2.2.1-cp36-none-any.whl size=100368 sha256=3dc7cbb3649913fa53e2aa283b08ac0e9996c1137a49383323461c44993a10d9
  Stored in directory: /root/.cache/pip/wheels/bb/a9/00/0078d26b0c15b31be0001af8eb659496709c361c69641303f1
Successfully built wfdb


In [0]:
import numpy as np
import sys, os
import wfdb
import pywt
import pickle as pk
from collections import Counter

In [0]:
data_names = ['100', '101', '102', '103', '104', '105', '106', '107', 
              '108', '109', '111', '112', '113', '114', '115', '116', 
              '117', '118', '119', '121', '122', '123', '124', '200', 
              '201', '202', '203', '205', '207', '208', '209', '210', 
              '212', '213', '214', '215', '217', '219', '220', '221', 
              '222', '223', '228', '230', '231', '232', '233', '234']

In [0]:
wid = 100

labels =  ['N', 'L', 'R', 'A', 'V']
X = []
Y = []
for d in data_names:
  r=wfdb.rdrecord('/content/drive/My Drive/ECG_project/files/mitdb/1.0.0/'+d)
  ann=wfdb.rdann('/content/drive/My Drive/ECG_project/files/mitdb/1.0.0/'+d, 'atr', return_label_elements=['label_store', 'symbol'])
  sig = np.array(r.p_signal[:,0])
  sig_len = len(sig)
  sym = ann.symbol
  pos = ann.sample
  beat_len = len(sym)
  for i in range(1,beat_len-1):
    if sym[i] in labels: 
      if (pos[i]-pos[i-1])>200 and (pos[i+1]-pos[i])>200:
        a = sig[pos[i]-150:pos[i]+150]
        a, cD3, cD2, cD1 = pywt.wavedec(a, 'db6', level=3)
        X.append(a)
        Y.append(labels.index(sym[i]))

In [0]:
X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)
print(Counter(Y))

(78546, 47)
(78546,)
Counter({0: 60621, 1: 7772, 2: 7141, 3: 1585, 4: 1427})


In [0]:
data_len = len(X)
np.random.seed(42)
idx = list(range(data_len))
np.random.shuffle(idx)

In [27]:
train_len = int(data_len*0.8) #
valid_len = int(data_len*0.2)

X_train = X[idx][:train_len]
X_valid = X[idx][train_len:train_len+valid_len]
Y_train = Y[idx][:train_len]
Y_valid = Y[idx][train_len:train_len+valid_len]


print(X_train.shape)
print(X_valid.shape)
print(Counter(Y_train))
print(Counter(Y_valid))

fn = "data_"+"NLRAV"+".pk"
with open(fn, "wb") as fw:
    pk.dump(X_train, fw, protocol=pk.HIGHEST_PROTOCOL)
    pk.dump(Y_train, fw, protocol=pk.HIGHEST_PROTOCOL)
    pk.dump(X_valid, fw, protocol=pk.HIGHEST_PROTOCOL)
    pk.dump(Y_valid, fw, protocol=pk.HIGHEST_PROTOCOL)

(62836, 47)
(15709, 47)
Counter({0: 48390, 1: 6280, 2: 5744, 3: 1280, 4: 1142})
Counter({0: 12231, 1: 1491, 2: 1397, 3: 305, 4: 285})


In [0]:
import numpy as np
import pickle as pk
import os, sys
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score 
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam

def get_session(gpu_fraction=0.1):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
    return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
  
K.set_session(get_session())



mode = "NLRAV" # NLRAV / NSVFQ
model_type = "CNN_GRU"
save = "result_NLRAV_0_1" # result_NLRAV_0

fn = "data_"+mode+".pk"
with open(fn, "rb") as fp:
    X_train = pk.load(fp)
    Y_train = pk.load(fp)
    X_valid = pk.load(fp)
    Y_valid = pk.load(fp)
    

if model_type != 'Dense':
    X_train = np.expand_dims(X_train, axis=-1)
    X_valid = np.expand_dims(X_valid, axis=-1)

print(X_train.shape)
print(X_valid.shape)
f_size = X_train.shape[1]
class_num = 5

#============================================#

lr = 0.005
batch_size=128

Y_train = keras.utils.to_categorical(Y_train, num_classes=class_num)

def make_model(model_type = 'LSTM'):
    model = Sequential()
    if model_type == 'LSTM':
        model.add(LSTM(64, return_sequences=True, dropout=0.1, input_shape=(f_size, 1)))
        model.add(LSTM(32, return_sequences=True, dropout=0.1))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
    elif model_type == 'GRU':
        #model.add(Input(256,256))
        model.add(GRU(32, dropout=0.5,recurrent_dropout=0.5,return_sequences=True))
        model.add(GRU(32, dropout=0.5,recurrent_dropout=0.5, return_sequences=True))
        model.add(Flatten())
        model.add(Dense(32))
        model.add(Dropout(0.5))
        model.add(Dense(10))
        #model.add(Activation('sigmoid', name='strong_out'))
    elif model_type == 'CNN_GRU':
        model.add(Conv1D(64, 3, activation='relu'))
        model.add(SpatialDropout1D(0.2))
        model.add(MaxPooling1D(4))
        model.add(CuDNNGRU(64))
        model.add(Dropout(0.1))

    model.add(Dense(class_num, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr))
    return model


(62836, 47, 1)
(15709, 47, 1)


In [0]:

model = make_model('GRU')
best_SE = 0
best_ACC = 0
best_model = make_model('GRU')
patience = 30
pcnt = 0

bin_label = lambda x: min(1,x)

for e in range(1, 3):

    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1, verbose=0)
    
    model.save_weights("weights_bilstm.h5")
    
    y_pred = model.predict(X_valid)
    y_pred = np.argmax(y_pred, axis=1)
    acc = np.sum(y_pred==Y_valid)/len(Y_valid)

    y_true = list(map(bin_label, Y_valid))
    y_pred = list(map(bin_label, y_pred))
    auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    SE = tp/(tp+fn)
    SP = tn/(fp+tn)

    if SE+acc > best_SE+best_ACC:
        best_SE, best_ACC = SE, acc
        best_model.set_weights(model.get_weights())
        pcnt = 0
    else:
        pcnt += 1
    
    print("Epoch: %d | SE: %.4f | Best SE: %.4f | ACC: %.4f | Best ACC: %.4f | AUC: %.4f | SP: %.4f" %(e, SE, best_SE, acc, best_ACC, auc, SP))

    


Epoch: 1 | SE: 0.7763 | Best SE: 0.7763 | ACC: 0.9304 | Best ACC: 0.9304 | AUC: 0.8835 | SP: 0.9907


KeyboardInterrupt: ignored

In [0]:
model.load_weights("weights_bilstm.h5")

In [28]:
X_train = X_train.squeeze()
X_train.shape

(62836, 47)

In [29]:
Y_train = Y_train.squeeze()
Y_train.shape

(62836,)

In [31]:
X_valid = X_valid.squeeze()
X_valid.shape

(15709, 47)

In [32]:
Y_valid= Y_valid.squeeze()
Y_valid.shape

(15709,)

In [0]:



# training a linear SVM classifier 
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, Y_train) 
 



In [0]:
svm_predictions = svm_model_linear.predict(X_valid)

# model accuracy for X_test   
accuracy = svm_model_linear.score(X_valid, Y_valid) 
  
# creating a confusion matrix 
cm = confusion_matrix(Y_valid, svm_predictions) 


In [39]:
print(cm)

[[12042   163    11     9     6]
 [  218  1267     0     1     5]
 [   60     1  1333     3     0]
 [   33     1     3   268     0]
 [   36    26    14     5   204]]


In [40]:
from sklearn.metrics import classification_report
print(classification_report(Y_valid, svm_predictions, target_names=labels))


              precision    recall  f1-score   support

           N       0.97      0.98      0.98     12231
           L       0.87      0.85      0.86      1491
           R       0.98      0.95      0.97      1397
           A       0.94      0.88      0.91       305
           V       0.95      0.72      0.82       285

    accuracy                           0.96     15709
   macro avg       0.94      0.88      0.91     15709
weighted avg       0.96      0.96      0.96     15709

